loading package and scenic object

In [1]:
# Required packages:
library(SCopeLoomR)
library(AUCell)
library(SCENIC)

# For some of the plots:
#library(dplyr)
library(KernSmooth)
library(RColorBrewer)
library(plotly)
library(BiocParallel)
library(grid)
library(ComplexHeatmap)
library(data.table)
library(gprofiler2)
library(stringr)
library(dplyr)
library(tibble)
library(ggrepel)
library(circlize)
library(svglite)


Attachement du package : ‘SCopeLoomR’


L'objet suivant est masqué depuis ‘package:base’:

    flush



Attachement du package : ‘SCENIC’


L'objet suivant est masqué depuis ‘package:AUCell’:

    plotEmb_rgb


KernSmooth 2.23 chargé
 Copyright M. P. Wand 1997-2009

Le chargement a nécessité le package : ggplot2


Attachement du package : ‘plotly’


L'objet suivant est masqué depuis ‘package:ggplot2’:

    last_plot


L'objet suivant est masqué depuis ‘package:stats’:

    filter


L'objet suivant est masqué depuis ‘package:graphics’:

    layout


ComplexHeatmap version 2.16.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite either one:
- Gu, Z. Complex Heatmap Visualization. iMeta 2022.
- Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
    genomic data. Bioinformati

In [2]:
getwd()

[1] "/work/user/adufour/notebook/scenic/visualisation_resultats/Microglia"

In [3]:
options(repr.plot.width = 18, repr.plot.height = 17, repr.plot.pointsize = 24)

In [4]:
scenicLoomPath <- file.path("/home/adufour/work/microglia/scenic_subcluster/work/f5/018827757f31bea0d1626d6b4dc4f6/scenic_visualize.loom")
motifEnrichmentFile <- file.path("/home/adufour/work/microglia/scenic_subcluster/out/scenic/10x_PBMC/multi_runs_cistarget/multi_runs_features_mtf.csv.gz")
file.exists(scenicLoomPath)
file.exists(motifEnrichmentFile)

[1] TRUE

[1] TRUE

In [5]:
loom <- open_loom(scenicLoomPath)
    # Read information from loom file:
    exprMat <- get_dgem(loom)
        exprMat_log <- log2(exprMat+1) # Better if it is logged/normalized
    regulons_incidMat <- get_regulons(loom, column.attr.name="Regulons")
        regulons <- regulonsToGeneLists(regulons_incidMat)
    regulonAUC <- get_regulons_AUC(loom, column.attr.name='RegulonsAUC')
    regulonAucThresholds <- get_regulon_thresholds(loom)
        regulonAucThresholds <- setNames(as.numeric(names(regulonAucThresholds)), regulonAucThresholds)
    embeddings <- get_embeddings(loom)
    cellInfo <- get_cell_annotation(loom)
close_loom(loom)

In [6]:
cellInfo = read.csv("/home/adufour/work/microglia/loom_metadata.csv")

Add subtype information

In [15]:
cellInfo$stim_cluster_pnd <-paste(cellInfo$STIM, cellInfo$new_clusters, cellInfo$PND, sep="_")

In [16]:
rownames(cellInfo) <- cellInfo$cell

In [ ]:
# Split the cells by stim_cluster_pnd:
cellsPerCat <- split(rownames(cellInfo), cellInfo[,"stim_cluster_pnd"])
cellsPerCat <- cellsPerCat[lengths(cellsPerCat) > 10] #remove categorie with too few cells
# Calculate average expression:
regulonActivity_byCat <- sapply(cellsPerCat, function(cells) rowMeans(getAUC(regulonAUC)[,cells]))

Prepare Heatmap

In [14]:
mat <- t(scale(t(regulonActivity_byCat[names(regulons),]), center = T, scale=T))
rownames(mat) <- gsub("_", " ", rownames(mat))
ha <- rowAnnotation(Regulons_size = anno_barplot(lengths(regulons), which = "row", width = unit(20, "mm")), annotation_name_gp = grid::gpar(fontsize=20), annotation_name_rot = 90, annotation_label = "Regulons size")
hb <- HeatmapAnnotation(foo = anno_block(gp = gpar(fill = rep(c("#6a9ad5", "#9078bf"),3)),
                                                   labels = rep(c('PBS', expression('IL-1-'*beta)), 3),
                                         labels_gp = gpar(col = "white", fontsize = 18)))
col_fun <- colorRamp2(seq(min(mat), max(mat), length = 100), colorRampPalette(rev(brewer.pal(n = 6, name = "RdYlBu")))(100))
lgd <- list(title = "Regulon activity", legend_height = unit(6, "cm"), grid_width = unit(1, "cm"), labels_gp = gpar(fontsize = 20), title_gp = gpar(fontsize = 18), title_position = "leftcenter-rot")
split = rep(1:6, each = 3)

In [15]:
column_order <- c("pbs_20_P1", "pbs_20_P3", "pbs_20_P5", "il1_20_P1", "il1_20_P3", "il1_20_P5", "pbs_21_P1", "pbs_21_P3", "pbs_21_P5", "il1_21_P1", "il1_21_P3", "il1_21_P5", "pbs_22_P1", "pbs_22_P3", "pbs_22_P5", "il1_22_P1", "il1_22_P3", "il1_22_P5")

In [16]:
options(repr.plot.width=30, repr.plot.height=20) # To set the figure size in Jupyter
pdf('/home/adufour/work/notebook/plots/Microglia/regulon_microglia_subcluster.pdf',width=20,height=22)
hm <- draw(ComplexHeatmap::Heatmap(mat[,column_order], col = col_fun, cluster_columns = FALSE, row_names_gp=grid::gpar(fontsize=24),column_names_gp=grid::gpar(fontsize=24),
                                   column_gap = unit(3, "mm"), right_annotation = ha, column_labels = rep(c("P1","P3","P5"),6), column_split = split, top_annotation = hb,
                                   column_title = NULL, heatmap_legend_param = lgd), padding = unit(c(20, 2, 2, 2), "mm"))
dev.off()

png 
  2

Write supplementary infos

In [12]:
y <- stack(regulons)
head(y)

,values,ind
,<chr>,<fct>
1,Actg1,Atf3_(+)
2,Adamts1,Atf3_(+)
3,Apoc1,Atf3_(+)
4,Apoe,Atf3_(+)
5,Atf3,Atf3_(+)
6,Btg2,Atf3_(+)


In [13]:
write.csv(y, "/home/adufour/work/table/regulon_microglia_subcluster.csv", row.names=FALSE)

In [20]:
write.csv(regulonActivity_byCat, "/home/adufour/work/table/regulon_AUC_microglia_subcluster.csv", row.names=TRUE)